# Mediapipe : Utilisation dans le cadre de la caractérisation d'une démarche

In [ ]:
#pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## Calcul des angles

In [2]:
def calculate_angles(a,b,c): #donc ici ça va etre notre angle abc genre b quoi
    a = np.array(a) #premier
    b = np.array(b) #deuxième
    c = np.array(c) #troisième
    
    #on va faire le calcul en radian
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0]) 
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
        return angle

## Compteur de pas

On enregistre la vidéo qui se trouvera sous le nom de gait_vid.avi (à ouvrir avec VLC). Cette vidéo comprend le squelette MediaPipe, l'angle du genou et le nombre de pas effectués.

In [35]:
def gait_video():
    
    video = cv2.VideoCapture(0)
    counter = 0 #compteur pour le comptage des mouvements
    stage = None

    if (video.isOpened() == False):
        print("Error reading video file")

    frame_width = int(video.get(3))
    frame_height = int(video.get(4))

    size = (frame_width, frame_height)

    result = cv2.VideoWriter('gait_vid.avi',cv2.VideoWriter_fourcc(*'MJPG'),10, size)


    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while video.isOpened():
            ret, frame = video.read() ## frame : video sans aucun filtre

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) ## image : video avec mediapipe
            image.flags.writeable = False

            #Make detection
            results = pose.process(image)

            #Recolor image to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #EXTRACT LANDMARKS
            try:
                landmarks = results.pose_landmarks.landmark

                #get coordinates
                hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee=[landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle=[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

                #calculate knee angle 
                angle = calculate_angles(hip, knee, ankle)

                #visualize angle and counter
                cv2.putText(image,str(angle) , tuple(np.multiply(knee,[640,480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,255,255),2,cv2.LINE_AA)
                cv2.putText(image,str(counter),(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,255,255),2,cv2.LINE_AA)
                #curl counter logic
                if angle < 160:
                    stage = "down"
                if angle > 160 and stage == "down":
                    stage = "up"
                    counter +=1

                #print(landmarks)
            except:
                pass
            
            #Render curl counter setup statux box
            cv2.rectangle(image,(0,0),(100,70),(245,117,16),-1)

            #Rep data
            cv2.putText(image,'REPS' , (30,15),
                        cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1, cv2.LINE_AA)

            cv2.putText(image,str(counter),
                        (40,50),
                        cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1, cv2.LINE_AA)

            #Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

            cv2.imshow('Gait + Mediapipe', image)

            if ret == True:
                #result.write(frame) ## choisir ici pour l'image sans le squelette Mediapipe
                result.write(image) ## on enregistre l'image avec le squelette Mediapipe
                #cv2.imshow('Gait', frame)

                if counter == 2 or cv2.waitKey(1) & 0xFF == ord('q'): ## on s'arrête si on a fait 2 pas ou arrêt forcé
                    #print("OK")
                    break

            else:
                print("KO")
                
                break


        video.release()
        result.release()
        cv2.destroyAllWindows()
        return counter

In [36]:
gait_video()

0

In [31]:
def cut_video():
    # couper la video en séquence de photos 
    
    vidcap = cv2.VideoCapture('gait_vid.avi')
    success,image = vidcap.read()
    count = 0
    while success:
        # si on voit un pied, on enregistre la frame
        cv2.imwrite("frame_%d.jpg" % count, image)     # save frame as JPEG file      
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    

In [32]:
cut_video()

In [ ]:

# a faire : tri des photos : choisir le 1er pas